In [1]:
import pandas as pd
import plotly.express as px
import os

# Configuración
pd.options.plotting.backend = "plotly"

# Rutas de archivos procesados
FILE_2000_2011 = '../data/processed/fondef_2000_2011_processed.csv'
FILE_2012_2017 = '../data/processed/fondef_2012_2017_processed.csv'

# Cargar datos
df_2000 = pd.read_csv(FILE_2000_2011)
df_2012 = pd.read_csv(FILE_2012_2017)

# Unificar para conteos
df_all = pd.concat([df_2000, df_2012], ignore_index=True)

print("Datos cargados exitosamente.")
print(f"Registros 2000-2011: {len(df_2000)}")
print(f"Registros 2012-2017: {len(df_2012)}")

Datos cargados exitosamente.
Registros 2000-2011: 591
Registros 2012-2017: 508


## 1. Montos Adjudicados por Área y Año (2000-2011)

Visualización de la inversión realizada por Fondef desglosada por las áreas temáticas inferidas.

In [2]:
# Filtrar datos con montos válidos
df_amounts = df_2000[df_2000['amount'] > 0].copy()

# Agrupar por Año y Área
amount_by_area_year = df_amounts.groupby(['year', 'area'])['amount'].sum().reset_index()

# Ordenar por año
amount_by_area_year = amount_by_area_year.sort_values('year')

# Crear Gráfico de Barras Apiladas
fig = px.bar(amount_by_area_year, 
             x="year", 
             y="amount", 
             color="area",
             title="Montos Adjudicados por Área y Año (2000-2011)",
             labels={
                 "year": "Año", 
                 "amount": "Monto Total (Miles $)", 
                 "area": "Área Temática"
             },
             template="plotly_white",
             text_auto='.2s')

fig.update_layout(
    xaxis_tickmode='linear',
    legend_title_text='Área',
    hovermode='x unified'
)

fig.show()

## 2. Cantidad de Proyectos por Área y Año (2000-2017)

Dado que no disponemos de los montos para el periodo 2012-2017, analizamos la **cantidad de proyectos adjudicados** para tener una visión completa de la evolución de las áreas prioritarias a lo largo de todo el periodo estudiado.

In [3]:
# Agrupar por Año y Área (Conteo)
count_by_area_year = df_all.groupby(['year', 'area']).size().reset_index(name='count')

# Filtrar años inválidos si los hubiera (ej. 0)
count_by_area_year = count_by_area_year[count_by_area_year['year'] > 1990]

# Crear Gráfico de Barras Apiladas (Conteo)
fig2 = px.bar(count_by_area_year, 
              x="year", 
              y="count", 
              color="area",
              title="Cantidad de Proyectos Adjudicados por Área y Año (2000-2017)",
              labels={
                  "year": "Año", 
                  "count": "Número de Proyectos", 
                  "area": "Área Temática"
              },
              template="plotly_white")

fig2.update_layout(
    xaxis_tickmode='linear',
    legend_title_text='Área',
    hovermode='x unified'
)

fig2.show()

## 3. Mapa de Calor: Intensidad de Inversión (2000-2011)

Una vista alternativa para identificar rápidamente los años y áreas con mayor financiamiento.

In [4]:
# Pivot table para heatmap
pivot_amounts = df_amounts.pivot_table(values='amount', index='area', columns='year', aggfunc='sum', fill_value=0)

fig3 = px.imshow(pivot_amounts,
                labels=dict(x="Año", y="Área", color="Monto"),
                x=pivot_amounts.columns,
                y=pivot_amounts.index,
                title="Mapa de Calor: Montos por Área y Año",
                aspect="auto",
                color_continuous_scale="Viridis")

fig3.update_layout(xaxis_tickmode='linear')
fig3.show()